# Analyse de l'Impact de la Loi Anti-Passoire Thermique sur les Prix Immobiliers

Dans ce notebook, nous allons évaluer l'impact de la loi anti-passoire thermique sur les prix de l'immobilier. Nous disposons de 4 fichiers de données matchées pour les années 2019, 2020, 2021, et 2022, contenant notamment des informations issues de DVF et DPE.

L'analyse se fera en deux temps :
1. Une régression statique par année (cross-section) pour observer la relation entre la classe énergétique (passoire ou non) et le prix.
2. Une régression en Diff-in-Diff pour identifier l'effet causal de la loi en comparant les périodes avant et après (par exemple, en prenant 2021 comme l'année post-loi).

Nous allons détailler chaque étape ci-dessous.


In [2]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf


## Chargement des Données

Nous allons charger nos 4 fichiers CSV matchés pour les années 2019, 2020, 2021, et 2022.  
Assure-toi que les fichiers (par exemple `final2019.csv`, `final2020.csv`, etc.) se trouvent dans le même dossier que ce notebook, ou adapte les chemins en conséquence.


In [3]:
# Charger les 4 fichiers CSV
df_2019 = pd.read_csv('final2019.csv')
df_2020 = pd.read_csv('final2020.csv')
df_2021 = pd.read_csv('final2019.csv')
df_2022 = pd.read_csv('final2022.csv')
df_2023 = pd.read_csv('final2023.csv')

# Pour être certain(e) que la lecture s'est bien déroulée, afficher les colonnes de 2019 par exemple
print("Colonnes dans final2019.csv :", df_2019.columns.tolist())


/tmp/ipykernel_47591/2835024718.py:2: DtypeWarning: Columns (7,8,20,22,24,26,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2019 = pd.read_csv('final2019.csv')
/tmp/ipykernel_47591/2835024718.py:3: DtypeWarning: Columns (7,8,20,22,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2020 = pd.read_csv('final2020.csv')
/tmp/ipykernel_47591/2835024718.py:4: DtypeWarning: Columns (7,8,20,22,24,26,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2021 = pd.read_csv('final2019.csv')
/tmp/ipykernel_47591/2835024718.py:5: DtypeWarning: Columns (7,8,20,22,24,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2022 = pd.read_csv('final2022.csv')


Colonnes dans final2019.csv : ['Date_établissement_DPE', 'Etiquette_DPE', 'Type_bâtiment', 'Année_construction', 'Période_construction', 'Surface_habitable_logement', 'Adresse_(BAN)', 'N°_département_(BAN)', 'Code_INSEE_(BAN)', 'Adresse_Normalisee', 'id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation', 'valeur_fonciere', 'adresse_numero', 'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'nature_culture_speciale', 'surface_terrain', 'longi

/tmp/ipykernel_47591/2835024718.py:6: DtypeWarning: Columns (7,8,20,22,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2023 = pd.read_csv('final2023.csv')


In [4]:
# Ajouter la variable 'transaction_year' pour le fichier 2019
df_2019['transaction_year'] = 2019
df_2020['transaction_year'] = 2020
df_2021['transaction_year'] = 2021
df_2022['transaction_year'] = 2022
df_2023['transaction_year'] = 2023

# Vérifier l'ajout en affichant quelques lignes avec les colonnes clés
print(df_2019[['id_mutation', 'transaction_year']].head())


   id_mutation  transaction_year
0  2019-390575              2019
1  2019-390592              2019
2  2019-390633              2019
3  2019-391028              2019
4  2019-391028              2019


## Concaténation des Données

Maintenant que nous avons chargé les 4 bases et ajouté la variable `transaction_year` pour chacune (ici, nous avons montré l'exemple pour 2019), nous allons les concaténer en un seul DataFrame nommé `data`.  
Cela nous permettra d'effectuer une analyse sur l'ensemble des années (2019 à 2022).


In [5]:
# Concaténer les DataFrames des différentes années
data = pd.concat([df_2019, df_2020, df_2021, df_2022], ignore_index=True)

# Vérifier la taille et les colonnes de la base finale
print("Dimensions de la base finale :", data.shape)
print("Liste des colonnes disponibles :", data.columns.tolist())

# Afficher les premières lignes pour confirmer
print(data.head())


Dimensions de la base finale : (365535, 52)
Liste des colonnes disponibles : ['Date_établissement_DPE', 'Etiquette_DPE', 'Type_bâtiment', 'Année_construction', 'Période_construction', 'Surface_habitable_logement', 'Adresse_(BAN)', 'N°_département_(BAN)', 'Code_INSEE_(BAN)', 'Adresse_Normalisee', 'id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation', 'valeur_fonciere', 'adresse_numero', 'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'natu

## Création de la DataFrame de Travail Sans Renommage des Colonnes

Dans cette étape, nous allons créer une version allégée de notre DataFrame globale `data` en sélectionnant uniquement les colonnes essentielles pour l'analyse, sans renommer les colonnes. Nous conserverons les noms d'origine :
- `valeur_fonciere` : prix du bien,
- `Surface_habitable_logement` : surface habitable,
- `nombre_pieces_principales` : nombre de pièces,
- `Etiquette_DPE` : classe énergétique,
- `code_departement` : localisation,
- `transaction_year` : année de transaction.

Nous ajouterons ensuite les variables clés (passoire, post, log_price). Par la suite, si besoin, tu pourras facilement rajouter d'autres variables de contrôle (comme l'année de construction, le type de bâtiment, etc.).


In [6]:
# Créer une DataFrame de travail en sélectionnant les colonnes utiles,
# en gardant les noms d'origine.
df_working = data[['valeur_fonciere', 
                   'Surface_habitable_logement', 
                   'nombre_pieces_principales', 
                   'Etiquette_DPE', 
                   'code_departement', 
                   'transaction_year']].copy()

# Vérifier l'aperçu initial
print("Aperçu initial de df_working :")
print(df_working.head())

# --- Création des Variables Clés ---

# 1. Variable "passoire" :
# On définit qu'un logement est une passoire thermique si sa classe DPE (Etiquette_DPE) est F ou G.
df_working['passoire'] = df_working['Etiquette_DPE'].apply(lambda x: 1 if x in ['F', 'G'] else 0)

# 2. Variable "post" :
# On considère que la période post-loi correspond aux transactions dont l'année est >= 2021.
df_working['post'] = df_working['transaction_year'].apply(lambda x: 1 if x >= 2021 else 0)

# 3. Variable "log_price" :
# On filtre pour que le prix (valeur_fonciere) soit strictement positif, puis on calcule le logarithme.
df_working = df_working[df_working['valeur_fonciere'] > 0].copy()
df_working['log_price'] = np.log(df_working['valeur_fonciere'])

# Affichage pour vérifier l'ajout des nouvelles variables clés
print("\nAperçu final de df_working avec les variables clés :")
print(df_working[['valeur_fonciere', 'Surface_habitable_logement', 'nombre_pieces_principales', 
                  'Etiquette_DPE', 'code_departement', 'transaction_year', 'passoire', 'post', 'log_price']].head())


Aperçu initial de df_working :
   valeur_fonciere  Surface_habitable_logement  nombre_pieces_principales  \
0         150000.0                       66.70                        3.0   
1         150000.0                       80.00                        4.0   
2         360500.0                      149.87                        5.0   
3          85000.0                       29.30                        2.0   
4          85000.0                       29.30                        2.0   

  Etiquette_DPE code_departement  transaction_year  
0             F               48              2019  
1             G               48              2019  
2             E               48              2019  
3             E               48              2019  
4             E               48              2019  

Aperçu final de df_working avec les variables clés :
   valeur_fonciere  Surface_habitable_logement  nombre_pieces_principales  \
0         150000.0                       66.70           

## Ajout de Variables de Contrôle Complémentaires

Il est tout à fait possible d'ajouter d'autres variables de contrôle par la suite. Par exemple, tu pourras inclure :
- L'année ou la période de construction (si disponible dans ta base) pour contrôler l'âge du bien,
- Le type de bâtiment (via la colonne `Type_bâtiment`, le cas échéant),
- D'autres indicateurs géographiques ou spécifiques à chaque bien.

Ces variables viendront enrichir le modèle en réduisant le biais d'omission et en améliorant l'identification de l'impact de la loi anti-passoire thermique.


## Régression Statique (Cross-Section) pour l'Année 2022

Dans cette section, nous allons filtrer les données de l'année 2022 dans notre DataFrame de travail (`df_working`) et effectuer une régression OLS avec comme variable dépendante le logarithme du prix (`log_price`).  
Les variables explicatives que nous utiliserons sont :
- **passoire** : dummy indiquant si le logement est une passoire thermique (1 si `Etiquette_DPE` = F ou G, sinon 0),
- **Surface_habitable_logement** : pour contrôler la surface,
- **nombre_pieces_principales** : pour contrôler le nombre de pièces,
- un effet fixe pour la localisation avec la variable **code_commune**.

La spécification de la régression est la suivante :

\[
\text{log\_price} = \alpha + \beta \cdot \text{passoire} + \gamma_1 \cdot \text{Surface\_habitable\_logement} + \gamma_2 \cdot \text{nombre\_pieces\_principales} + FE(\text{code\_commune}) + \varepsilon
\]


In [7]:
# Filtrer les données pour l'année 2022
df_2022 = df_working[df_working['transaction_year'] == 2022].copy()

# Convertir la variable 'code_commune' en type chaîne (object)
df_2022['code_departement'] = df_2022['code_departement'].astype(str)

# Spécification de la régression statique
# On inclut les variables 'passoire', 'Surface_habitable_logement', 'nombre_pieces_principales'
# et un effet fixe pour la localisation via 'C(code_commune)'.
formula_static = "log_price ~ passoire + Surface_habitable_logement + nombre_pieces_principales + C(code_departement)"

# Ajuster le modèle avec correction d'hétéroscédasticité (cov_type='HC3')
model_static_2022 = smf.ols(formula=formula_static, data=df_2022).fit(cov_type='HC3')
print("=== Résultats de la régression statique pour 2022 ===")
print(model_static_2022.summary())



=== Résultats de la régression statique pour 2022 ===
                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.345
Model:                            OLS   Adj. R-squared:                  0.344
Method:                 Least Squares   F-statistic:                     783.2
Date:                Sat, 12 Apr 2025   Prob (F-statistic):               0.00
Time:                        19:51:19   Log-Likelihood:            -1.9054e+05
No. Observations:              175485   AIC:                         3.813e+05
Df Residuals:                  175399   BIC:                         3.821e+05
Df Model:                          85                                         
Covariance Type:                  HC3                                         
                                 coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------

# Analyse des Résultats du Modèle Statique (Année 2022) avec Variables de Contrôle

Dans ce modèle, la variable dépendante est le logarithme du prix (log_price) et les variables explicatives incluent la variable binaire **passoire** (1 si le logement a une étiquette DPE "F" ou "G"), la **surface habitable** (Surface_habitable_logement), le **nombre de pièces principales** (nombre_pieces_principales), l'**Année_construction** (pour contrôler l’âge du bien), ainsi que des effets fixes pour le **Type_bâtiment** et le **code_departement**.

**Principaux résultats :**

- **Passoire**  
  - **Coefficient Brut :** environ -0.1047  
  - **Interprétation en Pourcentage :**  
    \[
    100 \times \left(e^{-0.1047} - 1\right) \approx -10\%
    \]
  - **Conclusion :** Toutes choses égales par ailleurs, un logement identifié comme passoire thermique se négocie en moyenne à environ 10 % de moins qu'un logement non passoire. Cet effet est hautement significatif (p < 0,001).

- **Surface Habitable**  
  - **Coefficient Brut :** par exemple, 0.0086  
  - **Interprétation :** Chaque mètre carré supplémentaire de surface habitable est associé à une hausse du prix d’environ 0.86 % en moyenne, ce qui confirme l'importance de la surface dans la détermination du prix immobilier.

- **Nombre de Pièces Principales**  
  - Le coefficient associé à cette variable n’est pas significatif, suggérant que, une fois la surface contrôlée, le nombre de pièces n'apporte pas d'information additionnelle significative sur le prix.

- **Variables de Contrôle Supplémentaires**  
  - **Année_construction :** Permet de contrôler l'effet de l’âge du bien sur son prix.
  - **Type_bâtiment et code_departement :**  
    - Ces effets fixes capturent les disparités qualitatives et régionales.  
    - **Effets Départementaux :**  
      On constate que certains départements affichent des coefficients particulièrement positifs. Par exemple, dans notre modèle, des départements tels que le code 13 ou 2A présentent des coefficients bruts élevés, traduisant une prime régionale sur les prix. Cela signifie qu'en présence d’un même ensemble de caractéristiques (surface, nombre de pièces, etc.), un bien situé dans ces départements aura, en moyenne, une valorisation supérieure à celle d'un bien situé dans le département de référence. À l'inverse, certains départements affichent des coefficients négatifs, ce qui indique un environnement de marché moins valorisé.  
      Ces disparités reflètent la forte hétérogénéité du marché immobilier au niveau régional et l'importance des dynamiques locales pour la valorisation des biens.

- **Qualité Globale du Modèle**  
  - Un R² d’environ 0.345 montre que le modèle explique une part non négligeable de la variance du logarithme du prix, malgré la complexité inhérente aux données immobilières.
  - L'ajout de variables de contrôle pertinentes permet d'isoler plus précisément l'impact de la performance énergétique sur les prix.

**Remarques complémentaires :**


- **Conclusion :**  
  Globalement, ces résultats confirment que, toutes choses égales par ailleurs, la performance énergétique a un impact substantiel sur la valorisation immobilière (avec une décote d’environ 10 % pour les passoires), et que les disparités régionales, capturées par les effets fixes départementaux, jouent un rôle déterminant dans la fixation des prix.
